In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
file_path = "data/overlap_category-V0.2.csv"
df = pd.read_csv(file_path, index_col=0)


In [3]:
overlap_matrix = df.to_numpy()

In [4]:
# Exclude the last column before summing
numeric_data = overlap_matrix[:, :-1]

# Convert the numeric part to integers (if necessary)
numeric_data = numeric_data.astype(int)

# Now sum along each row
S = np.sum(numeric_data, axis=1)


In [5]:
df_int = df[df["category"] == 'Internet Bank']

# Select only numeric columns
numeric_df = df_int.select_dtypes(include=[np.number])

# Calculate the sum across numeric columns for each row
df_int['Sum'] = numeric_df.sum(axis=1)

# Find the maximum sum value
max_sum = df_int['Sum'].max()
print("Maximum Sum:", max_sum)

Maximum Sum: 273


/tmp/ipykernel_159058/3714142479.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_int['Sum'] = numeric_df.sum(axis=1)


### The DataFrame is assumed to have:
####   - A "category" column (e.g., 'Internet Bank', 'Mega Bank', 'Hamrah Bank')
####   - Other columns (with numeric values) representing overlap scores among keywords.
####   - The index of the DataFrame holds the keyword names.


In [21]:
def compute_search_weights_by_group(df, mega_value, hamrah_value, internet_value):
    groups = {
        "Mega Bank": mega_value,
        "Hamrah Bank": hamrah_value,
        "Internet Bank": internet_value
    }
    
    result_dfs = []
    
    for group_name, user_value in groups.items():

        group_df = df[df["category"] == group_name].copy()
        
        group_keywords = group_df.index.tolist()
        
        group_overlap = group_df.loc[:, group_keywords]

       
        S_group = np.sum(group_overlap.to_numpy(), axis=1)
        print(S_group)

        S_group = group_overlap.sum(axis=1)
        print(S_group.iloc[0].values)
        
        alpha_index = np.argmax(S_group)
        alpha_keyword = group_keywords[alpha_index]
        
        overlaps_with_alpha = group_overlap[alpha_keyword].to_numpy()
        
        max_overlap = np.max(overlaps_with_alpha)
       
        if max_overlap > 0:
            final_weights = user_value - (user_value - 1) * (overlaps_with_alpha / max_overlap)
        else:
            final_weights = np.full_like(overlaps_with_alpha, user_value)
     
        final_weights = np.round(final_weights).astype(int)
        final_weights = np.clip(final_weights, 1, int(user_value))
      
        df_group_result = pd.DataFrame({
            "Keyword": group_keywords,
            "Raw Sum": S_group,
            "Alpha Keyword": alpha_keyword,  
            "Overlap with Alpha": overlaps_with_alpha,
            "Final Search Weight": final_weights,
            "Category": group_name
        })
        
        result_dfs.append(df_group_result)
    
    df_final = pd.concat(result_dfs, ignore_index=True)
  
    df_final = df_final.sort_values(by="Final Search Weight", ascending=False)
    return df_final

df_final = compute_search_weights_by_group(df, mega_value=8, hamrah_value=5, internet_value=10)
df_final.sort_values(["Category","Raw Sum"],inplace=True)
display(df_final)

[ 49 140 155 141 150 126 141 127 156  23]


AttributeError: 'numpy.int64' object has no attribute 'values'